In [1]:
import pandas as pd
from keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random
from scipy.integrate import quad
from scipy.interpolate import interp1d

ann_arr = load_model('RP_Arrival_time/Arrival_ANN4')
ann_end = load_model('RP_End_time/End_ANN4')
ann_chg = load_model('RP_Change_time/Change_ANN1')

#Section1
ann1 = load_model('RP_Section_1_new3/Section1_new3_ANN_2')
sc1 = pickle.load(open('RP_Section_1_new3/Section1_new3_scaler_ANN_2.pkl', 'rb'))
#Section2
'''ann2 = load_model('RP_Section_2_new2/Section2_new_ANN_2')
sc2 = pickle.load(open('RP_Section_2_new/Section2_new_scaler_ANN_2.pkl', 'rb'))
#Section3
ann3 = load_model('RP_Section_3_new/Section3_new_ANN_2')
sc3 = pickle.load(open('RP_Section_3_new/Section3_new_scaler_ANN_2.pkl', 'rb'))
#Section4
ann4 = load_model('RP_Section_4_new/Section4_new_ANN_7')
sc4 = pickle.load(open('RP_Section_4_new/Section4_new_scaler_ANN_7.pkl', 'rb'))
#Section5
ann5 = load_model('RP_Section_5_new/Section5_new_ANN_4')
sc5 = pickle.load(open('RP_Section_5_new/Section5_new_scaler_ANN_4.pkl', 'rb'))
#Section6
ann6 = load_model('RP_Section_6_new/Section6_new_ANN_2')
sc6 = pickle.load(open('RP_Section_6_new/Section6_new_scaler_ANN_2.pkl', 'rb'))'''

D:\Anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


"ann2 = load_model('RP_Section_2_new2/Section2_new_ANN_2')\nsc2 = pickle.load(open('RP_Section_2_new/Section2_new_scaler_ANN_2.pkl', 'rb'))\n#Section3\nann3 = load_model('RP_Section_3_new/Section3_new_ANN_2')\nsc3 = pickle.load(open('RP_Section_3_new/Section3_new_scaler_ANN_2.pkl', 'rb'))\n#Section4\nann4 = load_model('RP_Section_4_new/Section4_new_ANN_7')\nsc4 = pickle.load(open('RP_Section_4_new/Section4_new_scaler_ANN_7.pkl', 'rb'))\n#Section5\nann5 = load_model('RP_Section_5_new/Section5_new_ANN_4')\nsc5 = pickle.load(open('RP_Section_5_new/Section5_new_scaler_ANN_4.pkl', 'rb'))\n#Section6\nann6 = load_model('RP_Section_6_new/Section6_new_ANN_2')\nsc6 = pickle.load(open('RP_Section_6_new/Section6_new_scaler_ANN_2.pkl', 'rb'))"

In [2]:
# Perform adaptive quadrature integration
def integrate_with_AQ1(time, y):
    # Define a function for interpolation
    from scipy.interpolate import interp1d
    interp_func = interp1d(time, y, kind='cubic', fill_value="extrapolate")
    
    # Perform adaptive quadrature
    cumulative_integration = np.zeros_like(time)
    
    for i in range(1, len(time)):
        # Integrate from the start to the current time
        cumulative_integration[i], _ = quad(interp_func, time[0], time[i])
    
    return cumulative_integration

def integrate_with_AQ2(time, y):
    # Define a function for interpolation
    from scipy.interpolate import interp1d
    # Define an interpolation function for the data
    interp_func = interp1d(time, y, kind='cubic', fill_value="extrapolate")
    
    # Initialize cumulative integration array
    cumulative_integration = np.zeros_like(time)
    
    # Loop through each segment and calculate the integral
    for i in range(1, len(time)):
        # Integrate over the current segment [t_{i-1}, t_i]
        segment_integral, _ = quad(interp_func, time[i - 1], time[i], limit=50)

        # Accumulate the integral
        cumulative_integration[i] = cumulative_integration[i - 1] + segment_integral
    
    return cumulative_integration

# Perform trapezoidal rule integration
def integrate_with_TR(time, y):
    cumulative_integration = np.cumsum(np.gradient(time) * y)
    
    return cumulative_integration

# Function to perform integration
def perform_integration(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path)
    time_num = df['Time'].values  # Assuming time is in seconds and converting to milliseconds
    y_num = df['Pressure'].values  # Pressure units adjustment (kPa)
    max_time = np.max(time_num)
    cumulative_integration = integrate_with_TR(time_num, y_num)
    
    return max_time, cumulative_integration[-1]

# Define the function to generate time_ann and y_ann
def integrate_ann(Distance, Mass, Angle, termination_time, interval=0.01):

    arr = np.array ([[Mass,Distance, Angle]])
    arrival_time = ann_arr.predict(arr)
    end_time = ann_end.predict(arr)
    change_time = ann_chg.predict(arr)
    #print ("1",arrival_time,end_time, change_time)
    
    def stable_state(start, end, interval):    
        t = np.arange(start, end + interval, interval)
        t = t.reshape(len(t),1)
    
        p = np.repeat(0, len(t)).reshape(len(t),1)
        
        values = np.concatenate ((t, p), axis = 1)
        return values
    
    def p_fluctuation(d_set):
        d_set[:,0:4] = sc1.transform(d_set[:,0:4])
        p_f = ann1.predict(d_set)
        return p_f
    
    if termination_time < end_time:
        end_time = termination_time
        
    else:
        start = end_time
        end = termination_time
        values3 = stable_state(start, end, interval)
    #print ("2",arrival_time,end_time, Distance, Mass, Angle)
    t2= np.arange(arrival_time, end_time + interval, interval)
    t2 = t2.reshape(len(t2),1)
    
    if t2[0] < change_time:
        ph2 = np.array([[0, 1]])
    else:
        ph2 = np.array([[1, 0]])
        
    positive = np.array([0, 1])
    negative= np.array([1, 0])
    
    for time in t2[1:]:
        if time < change_time:
            ph2=np.vstack((ph2,positive))
        else:
            ph2=np.vstack((ph2,negative))
            
    m2 = np.repeat(Mass, len(t2)).reshape(len(t2),1)
    d2 = np.repeat(Distance, len(t2)).reshape(len(t2),1)
    a2 = np.repeat(Angle, len(t2)).reshape(len(t2),1)
    d_set = np.concatenate ((m2, d2, a2, t2, ph2), axis = 1)
    
    #p2 = p_fluctuation(Standoff_distance, d_set)
    p2 = p_fluctuation(d_set)
    p2 = p2.reshape((-1, 1))
    values2 = np.concatenate ((t2, p2), axis = 1)
    
    
    if 'values3' in globals():
        ml_model = np.concatenate((values2, values3), axis=0)
    else:
        ml_model = values2
        
    time = ml_model[:,0]
    y = ml_model[:,1]
    
    cumulative_integration = integrate_with_TR(time, y)
    
    return cumulative_integration[-1]


In [3]:
# Define the root directory
root_dir = Path('G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_new2')

# Mapping of file names to angles
angle_mapping1 = {'1.xlsx': 0, '2.xlsx': 15, '3.xlsx': 30, '4.xlsx': 45}
angle_mapping2 = {'1': 0, '2': 15, '3': 30, '4': 45}

# List to store file paths and corresponding metadata
file_data = []

# Traverse through all directories and subdirectories
for file_path in root_dir.rglob('*.xlsx'):
    # Extract the parent directory name
    parent_dir = file_path.parent.name
    
    # Parse distance and mass from the directory name
    distance, rest = parent_dir.split('m', 1)
    mass = rest.split('kg', 1)[0]
    
    # Extract the file name without extension and map to angle
    file_name = file_path.stem
    angle = angle_mapping1.get(file_name, None)  # Use None if file name is not in the mapping
    
    if angle is None:
        angle = angle_mapping2.get(file_name, None)
    
    # Append the data
    file_data.append({
        'path': str(file_path),
        'distance': float(distance),
        'mass': float(mass),
        'angle': angle,
        'file name': file_name
    })

# Convert the data to a DataFrame
sampled_files = pd.DataFrame(file_data)

# Randomly select 30% of the files
#sampled_files = sampled_files.sample(frac=0.3, random_state=40)
print (sampled_files)
# Apply integration to the sampled files and add a new column
sampled_files[['termination_time', 'num_integration']] = sampled_files['path'].apply(lambda x: pd.Series(perform_integration(x)))
# Apply ANN-based model to sampled files
sampled_files['ml_integration'] = sampled_files.apply(lambda row: integrate_ann(row['distance'], row['mass'], row['angle'], row['termination_time']),axis=1)

# Print the updated sampled files DataFrame
print(sampled_files)

# Optionally, save the updated DataFrame to a CSV for later use
sampled_files.to_excel('sampled_files_with_integration.xlsx', index=False)

                                                  path  distance  mass  angle  \
0    G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       1.5   0.5      0   
1    G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       1.5   0.5     15   
2    G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       1.5   0.5     30   
3    G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       1.5   0.5     45   
4    G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       1.5   2.5      0   
..                                                 ...       ...   ...    ...   
527  G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       3.0  34.5     45   
528  G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       3.0  35.0      0   
529  G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       3.0  35.0     15   
530  G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       3.0  35.0     30   
531  G:\Chamodi\LSDYNA3D\RP_TH_models\RP_Section_1_...       3.0  35.0     45   

    file name  
0          

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

file_path = 'sampled_files_with_integration.xlsx'
data = pd.read_excel(file_path, sheet_name= 'Sheet2')

data = data.sample(frac=0.3, random_state=40)

# Extract the numerical and ML model integration values
numerical_values = data['num_integration']
ml_model_values = data['ml_integration']

# Calculate MAE
mae = mean_absolute_error(numerical_values, ml_model_values)

# Calculate RMSE
rmse = mean_squared_error(numerical_values, ml_model_values, squared=False)

# Calculate R2
r2 = r2_score(numerical_values, ml_model_values)

# Print the results
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Coefficient of Determination (R^2): {r2:.4f}")

Mean Absolute Error (MAE): 358.0439
Root Mean Squared Error (RMSE): 481.7302
Coefficient of Determination (R^2): 0.7533
